In [2]:
import numpy as np
import cv2
import os
from scipy import ndimage
from scipy.spatial import distance
from sklearn.cluster import KMeans

In [64]:
filepath = './benchmark_data/20231106-182012'

In [65]:
def load_images_from_folder(folder):
    images = {}
    img_files = os.listdir(folder)
    for image_file in img_files:
        image_path = os.path.join(folder, image_file)
        if image_path == './benchmark_data/20231106-182012/676_-1.999885817130722_30.978629028113254_115.png':
            print("skipped one image so we can test with it")
            continue
        if os.path.isfile(image_path) and image_file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
            img = cv2.imread(image_path)
            if img is not None:
                images[image_file]=img
    return images

In [66]:
images = load_images_from_folder(filepath)

skipped one image so we can test with it


In [54]:
def sift_features(images):
    descriptor_list = []
    image_to_descriptors = {}
    sift = cv2.SIFT_create()

    for key, img in images.items():
        _kp, des = sift.detectAndCompute(img, None)
        if des is not None:
            descriptor_list.extend(des)
            image_to_descriptors[key] = des
    return descriptor_list, image_to_descriptors

descriptor_list, image_to_descriptors = sift_features(images) 

In [55]:
def unsupervised_kmeans(k, descriptor_list):
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(descriptor_list)
    visual_words = kmeans.cluster_centers_
    return visual_words

In [56]:
visual_words_unsupervised = unsupervised_kmeans(150, descriptor_list)

/Users/douglasdejesus/miniconda3/envs/game/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [57]:
def calculate_histogram(descriptors, visual_words):
    # Create an array to store the histogram
    histogram = np.zeros(len(visual_words))

    for descriptor in descriptors:
        # Find the nearest visual word for the descriptor
        nearest_word = np.argmin(np.linalg.norm(visual_words - descriptor, axis=1))
        # Increment the corresponding bin in the histogram
        histogram[nearest_word] += 1

    return histogram



In [58]:
# Calculate the target image's histogram using unsupervised visual words
# target_histogram = calculate_histogram(target_image_descriptors, visual_words_unsupervised)
# dataset_histograms = calculate_histogram(descriptor_list, visual_words_unsupervised)
image_to_histogram = {}
for key, desc in image_to_descriptors.items():
  hist = calculate_histogram(desc, visual_words_unsupervised)
  image_to_histogram[key] = hist

image_to_histogram


{'3891_1.4954626416029912_76.8280775890906_73.png': array([ 0.,  8.,  3.,  0.,  4.,  7.,  2.,  1.,  2.,  2.,  8.,  1., 15.,
         2., 10.,  0.,  4.,  0.,  3., 13.,  0.,  2.,  6.,  2.,  2.,  9.,
         0.,  2.,  0.,  2.,  0.,  0.,  0.,  5.,  6.,  0.,  1.,  1.,  1.,
         0.,  3.,  5., 16.,  6.,  0., 24.,  2.,  1.,  3.,  4.,  4.,  2.,
         3.,  0.,  1.,  1.,  6.,  3.,  1.,  4.,  2.,  2.,  0.,  1.,  3.,
         6.,  0.,  3.,  0.,  1.,  1.,  3.,  1.,  2.,  4.,  5.,  5.,  2.,
        10.,  5.,  4.,  1.,  1.,  0.,  6.,  5.,  2.,  1.,  2., 12.,  1.,
         0.,  3.,  4.,  2.,  5.,  1.,  9.,  7.,  1.,  5.,  2.,  2.,  5.,
         1.,  2.,  4.,  1.,  4.,  1.,  6.,  4.,  0.,  3.,  1.,  2.,  2.,
         1.,  3.,  1.,  0.,  1.,  1., 10.,  7.,  0.,  1.,  0.,  2.,  1.,
         3.,  4.,  2.,  3.,  3.,  1.,  8., 31.,  1.,  0.,  2.,  2.,  0.,
         0.,  1.,  9.,  3.,  9.,  5.,  1.]),
 '7291_42.53870045713994_72.54892788288909_12.png': array([1., 4., 0., 0., 0., 1., 6., 0., 0., 0., 0.

In [59]:
def match_target_image(target_histogram, dataset_histograms):
    best_match = None
    min_distance = float('inf')

    for key, histogram in dataset_histograms.items():
        distance = np.linalg.norm(target_histogram - histogram)
        if distance < min_distance:
            min_distance = distance
            best_match = key

    return best_match

In [62]:
query_image_path = './data/exploration_views/20231106-182012/676_-1.999885817130722_30.978629028113254_115.png'
query_img = cv2.imread(query_image_path)
sift = cv2.SIFT_create()
_kp, desc = sift.detectAndCompute(query_img, None)
query_hist = calculate_histogram(desc, visual_words_unsupervised)
query_hist


array([ 4.,  7.,  4.,  3.,  1.,  6.,  4.,  4.,  3.,  0.,  5.,  1.,  0.,
        5., 12.,  0.,  3.,  0.,  4.,  1.,  1.,  1.,  1.,  3.,  2.,  2.,
        1.,  2.,  2.,  7.,  1.,  0.,  0.,  9.,  1.,  0.,  0.,  4.,  5.,
        0.,  0.,  7.,  1.,  3.,  1.,  1.,  2.,  0.,  0.,  0.,  4.,  4.,
        0.,  0.,  0.,  1.,  5.,  3.,  2.,  2.,  1.,  3.,  1.,  0.,  1.,
        3.,  0.,  1.,  1.,  0.,  0.,  3.,  0.,  2.,  3.,  5.,  2.,  8.,
        0.,  4.,  1.,  1.,  1.,  0.,  5.,  0.,  3.,  0.,  1.,  1.,  1.,
        3.,  1.,  1.,  0.,  3.,  0.,  6.,  6.,  0.,  1.,  1.,  2.,  2.,
        5.,  2.,  0.,  0.,  3.,  4.,  1.,  4.,  1.,  5.,  2.,  3.,  1.,
        0.,  1.,  0.,  0.,  1.,  1.,  3.,  8.,  0.,  1.,  1.,  0.,  1.,
        1.,  3.,  0.,  2.,  1.,  1.,  2.,  0.,  1.,  0.,  2.,  2.,  1.,
        0.,  1.,  3.,  2.,  0.,  1.,  0.])

In [63]:
match = match_target_image(query_hist, image_to_histogram)
match

'701_-1.999885817130722_30.978629028113254_115.png'